In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import copy
import matplotlib.pyplot as plt

from tqdm import tqdm

In [12]:
with open('A4_640.npy', 'rb') as f:
    A4_PSFs_raw = np.load(f)
with open('A6_568.npy', 'rb') as f:
    A6_PSFs_raw = np.load(f)

print(len(A4_PSFs_raw))
print(len(A6_PSFs_raw))

113288
53738


In [24]:
# Reduce the size of the larger dataset (A4-640)
A4_indices = np.random.choice(range(len(A4_PSFs)), len(A6_PSFs))
A4_PSFs_reduced = A4_PSFs_raw[A4_indices]

# Combine, create labels, and reshuffle all
training_data = np.concatenate((A4_PSFs_reduced, A6_PSFs_raw))
labels = np.concatenate((np.zeros(len(A4_PSFs_reduced)), np.ones(len(A6_PSFs_raw))))

# Split into training, testing, and validation sets
training_percent = 0.8
testing_percent = 0.1
validation_percent = 0.1

random_seed = 1 # random seed for reproducibility
np.random.seed(random_seed)
all_indices = np.random.choice(np.arange(len(training_data)), size=len(training_data), replace=False)
print(all_indices)
print(len(all_indices))


[75143 56799 28987 ...  5192 77708 98539]
107476


In [ ]:
class PSFDataset(Dataset):
    def __init__(self, train):


In [ ]:
# model code
class CNN(nn.Module):
    # constructor
    def __init__(self, channel_widths, pooling, nonlinearity=nn.ReLU()):
        super(CNN, self).__init__()
        layers = []
        for i in range(len(channel_widths)-2):
            # convolution layer
            # you can play around with kernel_size, padding, and stride if you like
            # kernel_size is most likely to have an impact
            # arguments are: Conv2d(n_input_channels, n_output_channels, 
            #                       filter_side_length, padding, stride_length)
            layers.append(nn.Conv2d(channel_widths[i], channel_widths[i+1],
                                    kernel_size=5, padding=2, stride=1, bias=False))
            layers.append(nonlinearity) # non-linearity
        # add final layer
        layers.append(nn.Conv2d(channel_widths[-2], channel_widths[-1],
                                    kernel_size=5, padding=2, stride=1, bias=False))
        self.backbone = nn.Sequential(*layers)
        self.global_pooling = pooling # reduce each of the H x W feature maps to a single pooled value
        self.pool_size = pooling.output_size[0]*pooling.output_size[1]
        self.linear = nn.Linear(channel_widths[-1]*self.pool_size, 10)  # score each class to obtain logits
    # forward pass
    def forward(self, x):
        B = x.size(0) # number of input images
        features = self.backbone(x) # get feature maps (B, N_feature_maps, H, W)
        pooled_features = self.global_pooling(features) # (B, N_feature_maps, 1, 1)
        pooled_features = pooled_features.view(B, -1) # (B, N_feature_maps)
        logits = self.linear(pooled_features) # (B, N_classes)
        return logits